In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Attention, LayerNormalization, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

In [2]:
df = pd.read_csv('Traffic.csv')

In [3]:
df.shape

(2976, 9)

In [4]:
df.isna().sum()

,0
Time,0
Date,0
Day of the week,0
CarCount,0
BikeCount,0
BusCount,0
TruckCount,0
Total,0
Traffic Situation,0


In [5]:
df.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal


In [6]:
# Encode categorical "Traffic Situation"
label_encoder = LabelEncoder()
df["Traffic Situation Encoded"] = label_encoder.fit_transform(df["Traffic Situation"])

# Prepare features and labels
X = df[["CarCount", "BikeCount", "BusCount", "TruckCount", "Total"]].values
y = df["Traffic Situation Encoded"].values

# Reshape input for CNN
X = X.reshape((X.shape[0], X.shape[1], 1))


In [7]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
#build models

# AM-LSTM
input_am = Input(shape=(X.shape[1], 1))
lstm_out = LSTM(64, return_sequences=True)(input_am)
attention_out = Attention()([lstm_out, lstm_out])
attention_out = LayerNormalization()(attention_out)
context_vector = GlobalAveragePooling1D()(attention_out)
dense_am = Dense(64, activation='relu')(context_vector)
drop_am = Dropout(0.3)(dense_am)
output_am = Dense(len(np.unique(y)), activation='softmax')(drop_am)
am_lstm_model = Model(inputs=input_am, outputs=output_am)
am_lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# CNN Model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Train AM-LSTM
am_lstm_model.fit(X_train, y_train, epochs=20, batch_size=2, verbose=0)

In [10]:
# Train CNN
cnn_model.fit(X_train, y_train, epochs=20, batch_size=2, verbose=0)

In [11]:
# Get class probabilities from both models
probs_am = am_lstm_model.predict(X_test)
probs_cnn = cnn_model.predict(X_test)

# Average the predictions (soft voting)
ensemble_probs = (probs_am + probs_cnn) / 2
ensemble_preds = np.argmax(ensemble_probs, axis=1)

# Evaluate
ensemble_accuracy = accuracy_score(y_test, ensemble_preds)
ensemble_mae = mean_absolute_error(y_test, ensemble_preds)

print(f"\n Ensemble Model Accuracy: {ensemble_accuracy:.4f}")
print(f" Ensemble Model MAE: {ensemble_mae:.4f}")


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

 Ensemble Model Accuracy: 0.9430
 Ensemble Model MAE: 0.1040


In [12]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.0 MB/s eta 0:00:00


In [13]:
import gradio as gr

# Reverse label encoder for decoding predictions
reverse_traffic_labels = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))

# Dummy traffic pattern simulation (you can replace this with a model if available)
def simulate_traffic_counts(current_day, target_day, target_time):
    """
    Simulate traffic counts based on target day and time.
    You can replace this logic with an actual prediction model.
    """
    import random

    hour = int(target_time.split(":")[0])

    # Simulate traffic intensity pattern
    if hour in range(7, 10) or hour in range(17, 20):  # Rush hours
        base = [random.randint(20, 50), random.randint(30, 70), random.randint(5, 15), random.randint(5, 15)]
    elif hour in range(10, 17):  # Midday
        base = [random.randint(10, 30), random.randint(20, 50), random.randint(2, 8), random.randint(3, 10)]
    else:  # Night
        base = [random.randint(2, 10), random.randint(5, 15), random.randint(0, 3), random.randint(1, 5)]

    total = sum(base)
    return base + [total]

# Prediction function for Gradio
def predict_traffic(current_day, target_day, target_time):
    simulated_input = simulate_traffic_counts(current_day, target_day, target_time)
    X_input = np.array(simulated_input).reshape((1, 5, 1))

    probs_am = am_lstm_model.predict(X_input)
    probs_cnn = cnn_model.predict(X_input)
    ensemble_probs = (probs_am + probs_cnn) / 2
    ensemble_pred = np.argmax(ensemble_probs, axis=1)[0]

    return reverse_traffic_labels[ensemble_pred]

# Gradio Interface
gr.Interface(
    fn=predict_traffic,
    inputs=[
        gr.Dropdown(choices=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], label="Current Day of the Week"),
        gr.Dropdown(choices=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], label="Target Day of the Week"),
        gr.Textbox(label="Target Time (HH:MM, 24-hour format)")
    ],
    outputs=gr.Label(label="Predicted Traffic Status"),
    title="Long-Term Traffic Predictor",
    description="Predict traffic situation (Low, Medium, High) based on target day and time using ensemble of AM-LSTM and CNN."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5ff1486ca886ff316.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
